In [1]:
import time
import json
import ast
import os
import datetime
import io
from collections import defaultdict

import imageio
import boto3
import pandas as pd
import numpy as np
import imageio
import matplotlib.pyplot as plt

from brtdevkit.core.db.athena import AthenaClient
from brtdevkit.data import Dataset
from timezonefinder import TimezoneFinderL
import pytz
from brtdevkit.data import Dataset
from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import imageids_to_dataset, imageids_to_dataset_fast

from aletheia_dataset_creator.config.dataset_config import LEFT_CAMERAS, ALL_CAMERA_PAIRS_LIST
athena = AthenaClient()
s3 = boto3.resource('s3')


2023-11-17 13:58:16,086 - APIRequestor - ERROR - API error received | error_code : 403, error_message : {'extra': {'role_name': 'token_exchange'}, 'message': 'Role token_exchange is required to make this API call'}
ERROR:APIRequestor:API error received | error_code : 403, error_message : {'extra': {'role_name': 'token_exchange'}, 'message': 'Role token_exchange is required to make this API call'}


In [2]:
interesting_ids = '64ee6282325fbc2bb41d83e4','64d55c4aa12f47d52a8ab5eb'

In [3]:
query = f"""
SELECT
    id, camera_location, collected_on, group_id, robot_name
FROM image_jupiter
WHERE
    image_jupiter.id IN {interesting_ids}
LIMIT 1000
"""
print(query)
athena_df = athena.get_df(query)

INFO:botocore.tokens:Loading cached SSO token for brt



SELECT
    id, camera_location, collected_on, group_id, robot_name
FROM image_jupiter
WHERE
    image_jupiter.id IN ('64ee6282325fbc2bb41d83e4', '64d55c4aa12f47d52a8ab5eb')
LIMIT 1000



In [4]:
row = athena_df.iloc[0]
athena_df

,id,camera_location,collected_on,group_id,robot_name
0,64ee6282325fbc2bb41d83e4,T05,2023-08-18 20:30:52.059,1c4d5084583b42dabebf00a89e0ff270,halohitchhiker_101
1,64d55c4aa12f47d52a8ab5eb,T14,2023-08-01 18:42:09.228,fc62493e2bac4dfbb96835ee15291968,halohitchhiker_112


In [5]:
query = f"""
SELECT
    id, camera_location, collected_on, group_id, robot_name
FROM image_jupiter
WHERE
    camera_location = 'T08' AND
    robot_name = 'halohitchhiker_102'
LIMIT 10
"""
athena.get_df(query)

,id,camera_location,collected_on,group_id,robot_name


In [6]:
def same_robot_names(robot_name):
    subname = robot_name[:-1]
    return subname + '1', subname + '2', subname + '3'

In [7]:
dfs = []
for _, row in athena_df.iterrows():
    start_t = str(row["collected_on"] - datetime.timedelta(seconds=1.5)).split('.')[0]
    end_t = str(row["collected_on"] + datetime.timedelta(seconds=1.5)).split('.')[0]
    query = f"""
    SELECT
        id, camera_location, collected_on, group_id, robot_name
    FROM image_jupiter
    WHERE
        collected_on > TIMESTAMP'{start_t}' AND
        collected_on < TIMESTAMP'{end_t}' AND
        robot_name IN {same_robot_names(row["robot_name"])}
    LIMIT 1000
    """
    print(query)
    dfs.append(athena.get_df(query))



    SELECT
        id, camera_location, collected_on, group_id, robot_name
    FROM image_jupiter
    WHERE
        collected_on > TIMESTAMP'2023-08-18 20:30:50' AND
        collected_on < TIMESTAMP'2023-08-18 20:30:53' AND
        robot_name IN ('halohitchhiker_101', 'halohitchhiker_102', 'halohitchhiker_103')
    LIMIT 1000
    



    SELECT
        id, camera_location, collected_on, group_id, robot_name
    FROM image_jupiter
    WHERE
        collected_on > TIMESTAMP'2023-08-01 18:42:07' AND
        collected_on < TIMESTAMP'2023-08-01 18:42:10' AND
        robot_name IN ('halohitchhiker_111', 'halohitchhiker_112', 'halohitchhiker_113')
    LIMIT 1000
    


In [8]:
print(len(dfs[0]))
dfs[0].groupby('group_id').count()

60


,id,camera_location,collected_on,robot_name
group_id,,,,
1c4d5084583b42dabebf00a89e0ff270,8,8,8,8
2f6ad680a09f47e3968087d3b5debc57,4,4,4,4
40c2a1d3fa19426d89dc4465c9322319,4,4,4,4
6594ce45a9f24775ae6e94548dc58996,4,4,4,4
8f9debda16164cb39ab52c9c76620659,8,8,8,8
9a087914e83c464d8902714f6ff3e935,8,8,8,8
a7c91fd6ac2147e0b6fec7e17caaf0eb,8,8,8,8
c4878e04de84465681698b934b2ed3cf,8,8,8,8
d01704491241436db0641b2dc47ea925,8,8,8,8


In [21]:
both_seq = pd.concat([dfs[0].sort_values('collected_on'), dfs[1].sort_values('collected_on')], axis=0)

In [22]:
ids = list(dfs[0]['id']) + list(dfs[1]['id'])

In [23]:
print(len(dfs[0]['id']))
print(len(dfs[1]['id']))
print(len(ids))
print(','.join(ids))

60
60
120
64e6708690b41fcae138a4c1,64e6708d62698e86e530ca9c,64ee6285d977e860a16d4896,64e670cb43788d6f3d270930,64e670d508f3ae24fa546975,64e670c808f3ae24fa546946,64e67083ca3ea8d388b47ab7,64e68afa5e01413330ec12ca,64ee62a57257c880a79c6207,64e6708b880764ea65bbe195,64ee62adf36876c0c50e744f,64e670cff36876c0c508daee,64ee62e245f3f1852140b76d,64ee627af24655a2eddb8c49,64e67088a6c97b66e79ceda2,64e670d28db25bf27d0288a7,64e68b0a5e01413330ec12f4,64e68af88db25bf27d02b25f,64e670dda6c97b66e79cee94,64ee62d9f7f4625589496bf6,64e670da8db25bf27d0288c0,64ee62aa7257c880a79c6213,64e670c9f7f4625589443c8f,64ee62a17257c880a79c6201,64ee62dd880764ea65c3386d,64ee62ab880764ea65c337dc,64e68b0c43788d6f3d274904,64e670cd48ec6e07602feb35,64e670d7f36876c0c508db0d,64ee62db5e01413330f18904,64ee62a6b5783f9f881ebd27,64ee62d6b5783f9f881ebdf1,64ee62de22fae285992a5c9b,64ee62d8f36876c0c50e74ae,64ee6283880764ea65c3378c,64e68b104870a1d89e79801c,64ee627e2fea9c788d718e52,64e670d3880764ea65bbe281,64e670d943788d6f3d27095d,64e67089a6c97b6

In [24]:
from aletheia_dataset_creator.config import dataset_config
import itertools
halo_all_cams = []
for k, v in itertools.chain(dataset_config.HALO_CENTER_CAMERA_PAIRS.items(), dataset_config.HALO_NON_CENTER_CAMERA_PAIRS.items()):
    halo_all_cams.append(k)
    halo_all_cams.append(v)
halo_all_cams
new_dset_name = 'halo_all_cam_images_3_seconds_2_sequences'

['T02',
 'T03',
 'T06',
 'T07',
 'T10',
 'T11',
 'T14',
 'T15',
 'T01',
 'T03',
 'T02',
 'T04',
 'T05',
 'T07',
 'T06',
 'T08',
 'T09',
 'T11',
 'T10',
 'T12',
 'T13',
 'T15',
 'T14',
 'T16']

In [3]:
imageids_to_dataset_fast(both_seq, both_seq, dataset_name=new_dset_name, dataset_description="2 different 3 second image sequences with ALL camera data (left+right, front+rear+implement pod).")

In [8]:
dset = Dataset.retrieve(name=new_dset_name)
dset.download(f"{os.environ['DATASET_PATH']}/{new_dset_name}")

2023-11-17 14:13:38,554 - Dataset - INFO - start downloading dataset: 6556a116277289d6d58d8165 into /home/alexli/data/halo_all_cam_images_3_seconds_2_sequences
INFO:Dataset:start downloading dataset: 6556a116277289d6d58d8165 into /home/alexli/data/halo_all_cam_images_3_seconds_2_sequences
2023-11-17 14:13:39,080 - APIRequestor - ERROR - Timeout/Connection error, sleeping for 2 second(s)
ERROR:APIRequestor:Timeout/Connection error, sleeping for 2 second(s)
2023-11-17 14:13:41,462 - APIRequestor - ERROR - API error received | error_code : 403, error_message : {'extra': {'role_name': 'token_exchange'}, 'message': 'Role token_exchange is required to make this API call'}
ERROR:APIRequestor:API error received | error_code : 403, error_message : {'extra': {'role_name': 'token_exchange'}, 'message': 'Role token_exchange is required to make this API call'}
INFO:botocore.tokens:Loading cached SSO token for brt
INFO:botocore.tokens:SSO Token refresh succeeded
2023-11-17 14:13:47,724 - APIRequesto